In [1]:
{-# LANGUAGE TypeOperators #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold.Types
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup

import Control.Category.Constrained.Prelude
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2
viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

In [3]:
blockl, blockr :: Shade' (ℝ,ℝ)
[blockl,blockr] = [Shade' (x,1) (projector (0,1) ^+^ projector (1,0)) | x<-[0,2]]
plotWindow [plot blockl, plot blockr, viewRange]

GraphWindowSpecR2{lBound=-2.0, rBound=4.0, bBound=-1.0, tBound=3.0, xResolution=640, yResolution=480}

In [6]:
deq :: DifferentialEqn ℝ ℝ
deq (Shade' (_,y) δxy) = Shade' (denseLinear $ \δx -> y*δx/4)
                                (projector (denseLinear $ \δy -> metric δxy (0,δy)))

-- Option (Just (shyl', qvr)) = filterDEqnSolution_loc deq (blockl, [blockr])
-- (shyl', qvr $ 1)

In [7]:
itWhileJ :: (a -> Option a) -> a -> [a]
itWhileJ f x | Option (Just y) <- f x  = x : itWhileJ f y
itWhileJ _ _ = []

skipN :: Int -> [a] -> [a]
skipN n xs = case splitAt n xs of
    (x:_, xs') -> x : skipN n xs'
    ([], []) -> []

In [8]:
-- tf <- runRVar (uncrtFuncIntervalSpls 50 (-1,1) $ \x -> (1, exp (x/2))) StdRandom
-- plotWindow [plot tf, viewRange]
tf :: ℝ`Shaded`ℝ
tf = spanShading (\(Shade x _) -> 1:±[sqrt $ abs x]) $ fromLeafPoints [-2, -1 .. 2]

tfs = itWhileJ (filterDEqnSolution_static deq) tf
case length tfs of
 l | l>0 && l<100 -> plotWindow $ viewRange : (plot <$> take 2 tfs)
   | otherwise    -> error $ show l
-- onlyLeaves tf'

Global `Shade'` not implemented, so can't do intersection of zero co-shades.

GraphWindowSpecR2{lBound=-5.0, rBound=8.0, bBound=-3.0, tBound=5.0, xResolution=640, yResolution=480}